# Dataset preparation

In [28]:
from pathlib import Path
import json
import requests

from shutil import copyfile
from tqdm import tqdm_notebook
import xmltodict
from matplotlib import pyplot as plt
import pandas as pd

## Convert RectLabel format

In [8]:
alphabet = list('abcdefghijklmnopqrstuvwxyz') + ['na']

In [9]:
alphabet[:10]

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j']

In [10]:
atoi = {l: c for c, l in enumerate(alphabet)}

In [11]:
atoi['na']

26

In [12]:
PATH = Path('../data/')

In [13]:
YOLO_OUTPATH = PATH/'yolo'
YOLO_OUTPATH.mkdir(exist_ok=True)

In [14]:
VID_IDS = ['33qU9tQlEa4', '6iP3g_qSf28', 'EBH3wdRtZyg', 'pAz_mIjHglw']

In [15]:
# keep_ids = set(['h', 'e', 'a', 'l', 't', 'h', 'h', 'a', 'c', 'k'])

In [17]:
for vid_id in tqdm_notebook(VID_IDS):
    VID_PATH = PATH/'annotations'/vid_id
    for item in VID_PATH.iterdir():
        with open(item) as fd:
            doc = xmltodict.parse(fd.read())
            
        if 'object' not in doc['annotation']:
            continue
            
        class_name = doc['annotation']['object']['name']
            
        img_width = int(doc['annotation']['size']['width'])
        img_height = int(doc['annotation']['size']['height'])
        
        bnd_box = doc['annotation']['object']['bndbox']
        
        x_min, y_min, x_max, y_max = (
            int(bnd_box['xmin']), int(bnd_box['ymin']), int(bnd_box['xmax']), int(bnd_box['ymax']))
        width = x_max - x_min
        height = y_max - y_min

        class_id = atoi[class_name]
        
        video_file_id = str(item).split('/')[-1].split('.')[-2]
        copyfile(PATH/f'images/{vid_id}/{video_file_id}.jpg', YOLO_OUTPATH/f'{video_file_id}.jpg')
        with open(YOLO_OUTPATH/f'{video_file_id}.txt', 'w') as fh:
            fh.write(f'{class_id} {x_min / img_width} {y_min / img_height} {width / img_width} {height / img_height}')

## Convert DataTurks format

In [18]:
JSON_VID_IDS = ['ZO8Npgp4xvw', '_5NbYyUlcHU', 'hzxjWuly0Go']

In [21]:
for vid_id in tqdm_notebook(JSON_VID_IDS):
    for line in open(PATH/'annotations'/f'{vid_id}.json'):
        line = json.loads(line)
        anno = line['annotation'][0]

        xmin = min(
            anno['points'][0][0], anno['points'][1][0], anno['points'][2][0], anno['points'][3][0])

        ymin = min(
            anno['points'][0][1], anno['points'][1][1],
            anno['points'][2][1], anno['points'][3][1])

        xmax = max(
            anno['points'][0][0], anno['points'][1][0],
            anno['points'][2][0], anno['points'][3][0])
        ymax = max(
            anno['points'][0][1], anno['points'][1][1],
            anno['points'][2][1], anno['points'][3][1])
        
        width = xmax - xmin
        height = ymax - ymin
        
        class_id = atoi[anno['label']]
            
        url = line['content']
        
        video_file_id = url.split('.')[-2].split('__')[-1]

        response = requests.get(url)
        if response.status_code == 200:
            with open(YOLO_OUTPATH/f'{video_file_id}.jpg', 'wb') as f:
                f.write(response.content)
                print('Saved', YOLO_OUTPATH/f'{video_file_id}.jpg')

        with open(YOLO_OUTPATH/f'{video_file_id}.txt', 'w') as fh:
            fh.write(f'{class_id} {xmin} {ymin} {width} {height}')
            print('Saved', YOLO_OUTPATH/f'{video_file_id}.txt')

Saved ../data/yolo/_hzxjWuly0Go-frame-1011.jpg
Saved ../data/yolo/_hzxjWuly0Go-frame-1011.txt
Saved ../data/yolo/_hzxjWuly0Go-frame-1013.jpg
Saved ../data/yolo/_hzxjWuly0Go-frame-1013.txt
Saved ../data/yolo/_hzxjWuly0Go-frame-1016.jpg
Saved ../data/yolo/_hzxjWuly0Go-frame-1016.txt
Saved ../data/yolo/_hzxjWuly0Go-frame-1020.jpg
Saved ../data/yolo/_hzxjWuly0Go-frame-1020.txt
Saved ../data/yolo/_hzxjWuly0Go-frame-1021.jpg
Saved ../data/yolo/_hzxjWuly0Go-frame-1021.txt
Saved ../data/yolo/_hzxjWuly0Go-frame-1028.jpg
Saved ../data/yolo/_hzxjWuly0Go-frame-1028.txt
Saved ../data/yolo/_hzxjWuly0Go-frame-1032.jpg
Saved ../data/yolo/_hzxjWuly0Go-frame-1032.txt
Saved ../data/yolo/_hzxjWuly0Go-frame-1037.jpg
Saved ../data/yolo/_hzxjWuly0Go-frame-1037.txt
Saved ../data/yolo/_hzxjWuly0Go-frame-1039.jpg
Saved ../data/yolo/_hzxjWuly0Go-frame-1039.txt
Saved ../data/yolo/_hzxjWuly0Go-frame-1045.jpg
Saved ../data/yolo/_hzxjWuly0Go-frame-1045.txt
Saved ../data/yolo/_hzxjWuly0Go-frame-1046.jpg
Saved ../data

Saved ../data/yolo/_hzxjWuly0Go-frame-130.jpg
Saved ../data/yolo/_hzxjWuly0Go-frame-130.txt
Saved ../data/yolo/_hzxjWuly0Go-frame-1300.jpg
Saved ../data/yolo/_hzxjWuly0Go-frame-1300.txt
Saved ../data/yolo/_hzxjWuly0Go-frame-1306.jpg
Saved ../data/yolo/_hzxjWuly0Go-frame-1306.txt
Saved ../data/yolo/_hzxjWuly0Go-frame-1309.jpg
Saved ../data/yolo/_hzxjWuly0Go-frame-1309.txt
Saved ../data/yolo/_hzxjWuly0Go-frame-1312.jpg
Saved ../data/yolo/_hzxjWuly0Go-frame-1312.txt
Saved ../data/yolo/_hzxjWuly0Go-frame-1313.jpg
Saved ../data/yolo/_hzxjWuly0Go-frame-1313.txt
Saved ../data/yolo/_hzxjWuly0Go-frame-1315.jpg
Saved ../data/yolo/_hzxjWuly0Go-frame-1315.txt
Saved ../data/yolo/_hzxjWuly0Go-frame-1317.jpg
Saved ../data/yolo/_hzxjWuly0Go-frame-1317.txt
Saved ../data/yolo/_hzxjWuly0Go-frame-1322.jpg
Saved ../data/yolo/_hzxjWuly0Go-frame-1322.txt
Saved ../data/yolo/_hzxjWuly0Go-frame-1325.jpg
Saved ../data/yolo/_hzxjWuly0Go-frame-1325.txt
Saved ../data/yolo/_hzxjWuly0Go-frame-1330.jpg
Saved ../data/y

Saved ../data/yolo/_hzxjWuly0Go-frame-218.jpg
Saved ../data/yolo/_hzxjWuly0Go-frame-218.txt
Saved ../data/yolo/_hzxjWuly0Go-frame-219.jpg
Saved ../data/yolo/_hzxjWuly0Go-frame-219.txt
Saved ../data/yolo/_hzxjWuly0Go-frame-226.jpg
Saved ../data/yolo/_hzxjWuly0Go-frame-226.txt
Saved ../data/yolo/_hzxjWuly0Go-frame-229.jpg
Saved ../data/yolo/_hzxjWuly0Go-frame-229.txt
Saved ../data/yolo/_hzxjWuly0Go-frame-231.jpg
Saved ../data/yolo/_hzxjWuly0Go-frame-231.txt
Saved ../data/yolo/_hzxjWuly0Go-frame-237.jpg
Saved ../data/yolo/_hzxjWuly0Go-frame-237.txt
Saved ../data/yolo/_hzxjWuly0Go-frame-238.jpg
Saved ../data/yolo/_hzxjWuly0Go-frame-238.txt
Saved ../data/yolo/_hzxjWuly0Go-frame-246.jpg
Saved ../data/yolo/_hzxjWuly0Go-frame-246.txt
Saved ../data/yolo/_hzxjWuly0Go-frame-250.jpg
Saved ../data/yolo/_hzxjWuly0Go-frame-250.txt
Saved ../data/yolo/_hzxjWuly0Go-frame-252.jpg
Saved ../data/yolo/_hzxjWuly0Go-frame-252.txt
Saved ../data/yolo/_hzxjWuly0Go-frame-253.jpg
Saved ../data/yolo/_hzxjWuly0Go-fr

Saved ../data/yolo/_hzxjWuly0Go-frame-535.jpg
Saved ../data/yolo/_hzxjWuly0Go-frame-535.txt
Saved ../data/yolo/_hzxjWuly0Go-frame-542.jpg
Saved ../data/yolo/_hzxjWuly0Go-frame-542.txt
Saved ../data/yolo/_hzxjWuly0Go-frame-55.jpg
Saved ../data/yolo/_hzxjWuly0Go-frame-55.txt
Saved ../data/yolo/_hzxjWuly0Go-frame-56.jpg
Saved ../data/yolo/_hzxjWuly0Go-frame-56.txt
Saved ../data/yolo/_hzxjWuly0Go-frame-562.jpg
Saved ../data/yolo/_hzxjWuly0Go-frame-562.txt


IndexError: list index out of range

In [22]:
PATH = Path('../data')
(PATH/'yolo').mkdir(exist_ok=True)

In [23]:
alphabet = list('abcdefghijklmnopqrstuvwxyz') + ['na']
atoi = {l: c for c, l in enumerate(alphabet)}
itoa = {c: l for c, l in enumerate(alphabet)}

In [29]:
imgs = []
classes = []
bbs = []

from tqdm import tqdm
import math


for p in tqdm((PATH/'yolo').iterdir()):
    fn = p.parts[-1]

    if fn.endswith('.jpg'):
        imgs.append(fn)
        height, width, channels = plt.imread(p).shape

        labels = open(str(p).replace('.jpg', '.txt')).read().split()
        cls = itoa[int(labels[0])]
        xmin, ymin, bb_width, bb_height = labels[1:]
        
        bb_width_scaled = float(bb_width) * float(width)
        bb_height_scaled = float(bb_height) * float(height)
        
        # Have to unscale from Yolo format
        x = int(round(float(xmin) * float(width)))
        
        y = int(round(float(ymin) * float(height)))
                
        x2 = int(round(x + bb_width_scaled))
        y2 = int(round(y + bb_height_scaled))
            
        # Dunno why Fast.ai format has to be like this
        bb = f'{y} {x} {y2} {x2}'
        
        bbs.append(bb)
        classes.append(cls)

2894it [00:35, 80.96it/s] 


In [30]:
classes_df = pd.DataFrame({'image': imgs, 'class': classes})[['image', 'class']]
bbs_df = pd.DataFrame({'image': imgs, 'bb': bbs})[['image', 'bb']]

In [31]:
classes_df.to_csv(PATH/'train_classes.csv', index=False, columns=['image', 'class'])
bbs_df.to_csv(PATH/'train_bbs.csv', index=False, columns=['image', 'bb'])